In [3]:
%load_ext autoreload
%autoreload 2
import os;
import pandas as pd
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import socceraction.atomic.vaep.features as fs
import socceraction.atomic.vaep.labels as lab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
## Configure file and folder names
datafolder = "../data-fifa"
spadl_h5 = os.path.join(datafolder,"atomic-spadl-statsbomb.h5")
features_h5 = os.path.join(datafolder,"atomic-features.h5")
labels_h5 = os.path.join(datafolder,"atomic-labels.h5")
predictions_h5 = os.path.join(datafolder,"atomic-predictions-one-action.h5")

In [7]:
games = pd.read_hdf(spadl_h5,"games")
traingames = games
testgames = games
print("nb of games:", len(traingames),len(testgames))

nb of games: 64 64


In [8]:
# 1. Select feature set X
xfns = [#fs.actiontype,
       fs.actiontype_onehot,
       #fs.bodypart,
       fs.bodypart_onehot,
       fs.goalscore,
       fs.location,
       fs.polar,
       fs.direction,
       fs.team,
       fs.time,
       fs.time_delta
      ]
nb_prev_actions = 1

Xcols = fs.feature_column_names(xfns,nb_prev_actions)

def getXY(games,Xcols):
    # generate the columns of the selected feature
    X = []
    for game_id in tqdm.tqdm(games.game_id,desc="selecting features"):
        Xi = pd.read_hdf(features_h5,f"game_{game_id}")
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    # 2. Select label Y
    Ycols = ["scores","concedes"]
    Y = []
    for game_id in tqdm.tqdm(games.game_id,desc="selecting label"):
        Yi = pd.read_hdf(labels_h5,f"game_{game_id}")
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    return X,Y

X,Y = getXY(traingames,Xcols)
print("X:", list(X.columns))
print("Y:", list(Y.columns))
X = X.fillna(0)

selecting label: 100%|██████████| 64/64 [00:00<00:00, 93.12it/s]


X: ['type_pass_a0', 'type_cross_a0', 'type_throw_in_a0', 'type_freekick_crossed_a0', 'type_freekick_short_a0', 'type_corner_crossed_a0', 'type_corner_short_a0', 'type_take_on_a0', 'type_foul_a0', 'type_tackle_a0', 'type_interception_a0', 'type_shot_a0', 'type_shot_penalty_a0', 'type_shot_freekick_a0', 'type_keeper_save_a0', 'type_keeper_claim_a0', 'type_keeper_punch_a0', 'type_keeper_pick_up_a0', 'type_clearance_a0', 'type_bad_touch_a0', 'type_non_action_a0', 'type_dribble_a0', 'type_goalkick_a0', 'type_receival_a0', 'type_out_a0', 'type_offside_a0', 'type_goal_a0', 'type_owngoal_a0', 'type_yellow_card_a0', 'type_red_card_a0', 'type_corner_a0', 'type_freekick_a0', 'bodypart_foot_a0', 'bodypart_head_a0', 'bodypart_other_a0', 'goalscore_team', 'goalscore_opponent', 'goalscore_diff', 'x_a0', 'y_a0', 'dist_to_goal_a0', 'angle_to_goal_a0', 'dx_a0', 'dy_a0', 'period_id_a0', 'time_seconds_a0', 'time_seconds_overall_a0']
Y: ['scores', 'concedes']


In [37]:
%%time
# train classifiers F(X) = Y
import xgboost
#from sklearn.linear_model import LogisticRegression

Y_hat = pd.DataFrame()
models = {}
for col in list(Y.columns):
    print(col)
    model = xgboost.XGBClassifier(n_estimators=50,max_depth=3,n_jobs=-3,verbosity=2)
    #model = LogisticRegression(solver="lbfgs")
    model.fit(X,Y[col])
    models[col] = model

scores
[22:15:44] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:45] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:46] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:46] INFO: /workspace/src/tree

[22:15:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 10 extra nodes, 0 pruned nodes, max_depth=3
[22:15:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[22:15:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:59] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:15:59] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[22:15:59] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:15:59] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[22:15:59] INFO: /workspace/src/tree/upda

In [38]:
testX,testY = X,Y

In [41]:
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss

def evaluate(y,y_hat):
    p = sum(y)/len(y)
    base = [p] * len(y)
    brier = brier_score_loss(y,y_hat)
    print(f"  Brier score: %.5f (%.5f)" % (brier,brier/brier_score_loss(y,base)))
    ll = log_loss(y,y_hat)
    print(f"  log loss score: %.5f (%.5f)" % (ll,ll/log_loss(y,base)))
    print(f"  ROC AUC: %.5f" % roc_auc_score(y,y_hat))

for col in testY.columns:
    Y_hat[col] = [p[1] for p in models[col].predict_proba(testX)]
    print(f"### Y: {col} ###")
    evaluate(testY[col],Y_hat[col])

### Y: scores ###
  Brier score: 0.00560 (0.73363)
  log loss score: 0.02659 (0.58941)
  ROC AUC: 0.93521
### Y: concedes ###
  Brier score: 0.00109 (0.61458)
  log loss score: 0.00605 (0.46503)
  ROC AUC: 0.95919


### Save predictions

In [42]:
# get rows with game id per action
A = []
for game_id in tqdm.tqdm(testgames.game_id,"loading game ids"):
    Ai = pd.read_hdf(spadl_h5,f"atomic_actions/game_{game_id}")
    A.append(Ai[["game_id"]])
A = pd.concat(A)
A = A.reset_index(drop=True)

# concatenate action game id rows with predictions and save per game
grouped_predictions = pd.concat([A,Y_hat],axis=1).groupby("game_id")
for k,df in tqdm.tqdm(grouped_predictions,desc="saving predictions per game"):
    df = df.reset_index(drop=True)
    df[Y_hat.columns].to_hdf(predictions_h5,f"game_{int(k)}")

saving predictions per game: 100%|██████████| 64/64 [00:01<00:00, 52.21it/s]
